In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import os

In [2]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.middle = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        middle = self.middle(encoded)
        decoded = self.decoder(middle)
        return decoded


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [3]:
def train(generator, discriminator, train_loader, num_epochs, device):
    criterion_GAN = nn.MSELoss()
    criterion_cycle = nn.L1Loss()
    criterion_identity = nn.L1Loss()

    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for batch_idx, (bw_imgs, color_imgs) in enumerate(train_loader):
            bw_imgs = bw_imgs.to(device)
            color_imgs = color_imgs.to(device)

            # Train the generator
            optimizer_G.zero_grad()

            fake_color_imgs = generator(bw_imgs)

            # Identity loss
            resized_color_imgs = F.interpolate(color_imgs, size=fake_color_imgs.size()[2:], mode='bilinear', align_corners=True)
            loss_identity = criterion_identity(fake_color_imgs, resized_color_imgs) * 5.0

            # GAN loss
            pred_fake = discriminator(fake_color_imgs)
            loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))

            # Cycle loss
            recovered_bw_imgs = generator(fake_color_imgs)
            loss_cycle = criterion_cycle(recovered_bw_imgs, bw_imgs) * 10.0

            # Total generator loss
            loss_G = loss_identity + loss_GAN + loss_cycle
            loss_G.backward()
            optimizer_G.step()

            # Train the discriminator
            optimizer_D.zero_grad()

            pred_real = discriminator(color_imgs)
            loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

            pred_fake = discriminator(fake_color_imgs.detach())
            loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

            # Total discriminator loss
            loss_D = (loss_real + loss_fake) * 0.5
            loss_D.backward()
            optimizer_D.step()

            
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss G: {loss_G.item()}, Loss D: {loss_D.item()}")

In [4]:
def preprocess_image(image_path, size):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transform(image)

def load_dataset(bw_folder, color_folder, image_size, batch_size):
    bw_image_paths = [os.path.join(bw_folder, filename) for filename in os.listdir(bw_folder) if not filename.startswith('.')]
    color_image_paths = [os.path.join(color_folder, filename) for filename in os.listdir(color_folder) if not filename.startswith('.')]

    dataset = [(preprocess_image(bw_path, image_size), preprocess_image(color_path, image_size)) for bw_path, color_path in zip(bw_image_paths, color_image_paths)]
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_loader

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = (256, 256)
batch_size = 8
num_epochs = 200

bw_folder = "black_and_white"
color_folder = "colored"

train_loader = load_dataset(bw_folder, color_folder, image_size, batch_size)

generator = Generator().to(device)
discriminator = Discriminator().to(device)

train(generator, discriminator, train_loader, num_epochs, device)

Epoch 1/200, Batch 0/17, Loss G: 13.09152603149414, Loss D: 0.2650907337665558
Epoch 1/200, Batch 1/17, Loss G: 12.609439849853516, Loss D: 0.23490405082702637
Epoch 1/200, Batch 2/17, Loss G: 12.114734649658203, Loss D: 0.2103501260280609
Epoch 1/200, Batch 3/17, Loss G: 11.147067070007324, Loss D: 0.2101161628961563
Epoch 1/200, Batch 4/17, Loss G: 10.430885314941406, Loss D: 0.2173001766204834
Epoch 1/200, Batch 5/17, Loss G: 9.90399169921875, Loss D: 0.17615407705307007
Epoch 1/200, Batch 6/17, Loss G: 9.613513946533203, Loss D: 0.16853740811347961
Epoch 1/200, Batch 7/17, Loss G: 9.433649063110352, Loss D: 0.17952758073806763
Epoch 1/200, Batch 8/17, Loss G: 9.1299409866333, Loss D: 0.1319616734981537
Epoch 1/200, Batch 9/17, Loss G: 8.888096809387207, Loss D: 0.15965761244297028
Epoch 1/200, Batch 10/17, Loss G: 8.867410659790039, Loss D: 0.12664148211479187
Epoch 1/200, Batch 11/17, Loss G: 8.708919525146484, Loss D: 0.11643802374601364
Epoch 1/200, Batch 12/17, Loss G: 8.909759

Epoch 7/200, Batch 1/17, Loss G: 6.432782173156738, Loss D: 0.06283333152532578
Epoch 7/200, Batch 2/17, Loss G: 6.343683242797852, Loss D: 0.09160955250263214
Epoch 7/200, Batch 3/17, Loss G: 6.814825057983398, Loss D: 0.1115419790148735
Epoch 7/200, Batch 4/17, Loss G: 6.3721923828125, Loss D: 0.11993217468261719
Epoch 7/200, Batch 5/17, Loss G: 6.58156681060791, Loss D: 0.1399574726819992
Epoch 7/200, Batch 6/17, Loss G: 6.355290412902832, Loss D: 0.13299697637557983
Epoch 7/200, Batch 7/17, Loss G: 6.118590354919434, Loss D: 0.16448067128658295
Epoch 7/200, Batch 8/17, Loss G: 6.687137126922607, Loss D: 0.12190774828195572
Epoch 7/200, Batch 9/17, Loss G: 6.188803672790527, Loss D: 0.09463074803352356
Epoch 7/200, Batch 10/17, Loss G: 6.4184746742248535, Loss D: 0.15777212381362915
Epoch 7/200, Batch 11/17, Loss G: 6.660912990570068, Loss D: 0.24760396778583527
Epoch 7/200, Batch 12/17, Loss G: 6.237063407897949, Loss D: 0.07776187360286713
Epoch 7/200, Batch 13/17, Loss G: 6.10705

Epoch 13/200, Batch 1/17, Loss G: 5.402318000793457, Loss D: 0.1356886923313141
Epoch 13/200, Batch 2/17, Loss G: 6.151425361633301, Loss D: 0.032498106360435486
Epoch 13/200, Batch 3/17, Loss G: 5.6973466873168945, Loss D: 0.07681769877672195
Epoch 13/200, Batch 4/17, Loss G: 5.655092239379883, Loss D: 0.06450702995061874
Epoch 13/200, Batch 5/17, Loss G: 5.887468338012695, Loss D: 0.0416824072599411
Epoch 13/200, Batch 6/17, Loss G: 5.93261194229126, Loss D: 0.05630677938461304
Epoch 13/200, Batch 7/17, Loss G: 5.813591003417969, Loss D: 0.07484953850507736
Epoch 13/200, Batch 8/17, Loss G: 5.486230850219727, Loss D: 0.12387014180421829
Epoch 13/200, Batch 9/17, Loss G: 6.343947410583496, Loss D: 0.04154100641608238
Epoch 13/200, Batch 10/17, Loss G: 6.136608123779297, Loss D: 0.14904572069644928
Epoch 13/200, Batch 11/17, Loss G: 5.536066055297852, Loss D: 0.2080211341381073
Epoch 13/200, Batch 12/17, Loss G: 6.230137825012207, Loss D: 0.11286080628633499
Epoch 13/200, Batch 13/17, 

Epoch 19/200, Batch 0/17, Loss G: 5.202698707580566, Loss D: 0.11549351364374161
Epoch 19/200, Batch 1/17, Loss G: 5.72239351272583, Loss D: 0.07175226509571075
Epoch 19/200, Batch 2/17, Loss G: 5.584817886352539, Loss D: 0.031313762068748474
Epoch 19/200, Batch 3/17, Loss G: 5.622849464416504, Loss D: 0.02048611454665661
Epoch 19/200, Batch 4/17, Loss G: 5.316562652587891, Loss D: 0.04540642723441124
Epoch 19/200, Batch 5/17, Loss G: 5.4059858322143555, Loss D: 0.0681653693318367
Epoch 19/200, Batch 6/17, Loss G: 5.6468000411987305, Loss D: 0.09095162898302078
Epoch 19/200, Batch 7/17, Loss G: 5.202019691467285, Loss D: 0.1755920946598053
Epoch 19/200, Batch 8/17, Loss G: 5.995543479919434, Loss D: 0.15092217922210693
Epoch 19/200, Batch 9/17, Loss G: 5.239152908325195, Loss D: 0.07461901754140854
Epoch 19/200, Batch 10/17, Loss G: 5.350446701049805, Loss D: 0.17046640813350677
Epoch 19/200, Batch 11/17, Loss G: 5.950477600097656, Loss D: 0.13114793598651886
Epoch 19/200, Batch 12/17,

Epoch 24/200, Batch 16/17, Loss G: 5.774815559387207, Loss D: 0.05940798670053482
Epoch 25/200, Batch 0/17, Loss G: 5.593822479248047, Loss D: 0.08447359502315521
Epoch 25/200, Batch 1/17, Loss G: 5.340854644775391, Loss D: 0.03468197584152222
Epoch 25/200, Batch 2/17, Loss G: 5.130141735076904, Loss D: 0.053131457418203354
Epoch 25/200, Batch 3/17, Loss G: 5.364422798156738, Loss D: 0.0724051371216774
Epoch 25/200, Batch 4/17, Loss G: 4.948331832885742, Loss D: 0.11713887006044388
Epoch 25/200, Batch 5/17, Loss G: 4.768622398376465, Loss D: 0.23112693428993225
Epoch 25/200, Batch 6/17, Loss G: 5.704066276550293, Loss D: 0.09375753253698349
Epoch 25/200, Batch 7/17, Loss G: 5.53560733795166, Loss D: 0.0822746604681015
Epoch 25/200, Batch 8/17, Loss G: 4.980512619018555, Loss D: 0.07486200332641602
Epoch 25/200, Batch 9/17, Loss G: 5.308040618896484, Loss D: 0.02640945464372635
Epoch 25/200, Batch 10/17, Loss G: 5.41053581237793, Loss D: 0.03141820430755615
Epoch 25/200, Batch 11/17, Lo

...

Epoch 177/200, Batch 13/17, Loss G: 3.63297176361084, Loss D: 0.006844365503638983
Epoch 177/200, Batch 14/17, Loss G: 3.638988733291626, Loss D: 0.009095540270209312
Epoch 177/200, Batch 15/17, Loss G: 3.8226613998413086, Loss D: 0.012933354824781418
Epoch 177/200, Batch 16/17, Loss G: 3.9920172691345215, Loss D: 0.004048631060868502
Epoch 178/200, Batch 0/17, Loss G: 3.67984676361084, Loss D: 0.013127915561199188
Epoch 178/200, Batch 1/17, Loss G: 3.9353837966918945, Loss D: 0.005528673063963652
Epoch 178/200, Batch 2/17, Loss G: 4.004917144775391, Loss D: 0.003610357642173767
Epoch 178/200, Batch 3/17, Loss G: 3.802338123321533, Loss D: 0.004233046900480986
Epoch 178/200, Batch 4/17, Loss G: 3.676161766052246, Loss D: 0.004252527374774218
Epoch 178/200, Batch 5/17, Loss G: 3.7315502166748047, Loss D: 0.006973654497414827
Epoch 178/200, Batch 6/17, Loss G: 3.955984115600586, Loss D: 0.005387216340750456
Epoch 178/200, Batch 7/17, Loss G: 3.7193875312805176, Loss D: 0.0079052858054637

Epoch 183/200, Batch 9/17, Loss G: 3.8344030380249023, Loss D: 0.007082946598529816
Epoch 183/200, Batch 10/17, Loss G: 3.7452945709228516, Loss D: 0.0035603628493845463
Epoch 183/200, Batch 11/17, Loss G: 3.643712282180786, Loss D: 0.006512570194900036
Epoch 183/200, Batch 12/17, Loss G: 3.7464354038238525, Loss D: 0.00449999887496233
Epoch 183/200, Batch 13/17, Loss G: 3.737161636352539, Loss D: 0.008667769841849804
Epoch 183/200, Batch 14/17, Loss G: 3.649843692779541, Loss D: 0.012268942780792713
Epoch 183/200, Batch 15/17, Loss G: 3.835559844970703, Loss D: 0.002682237885892391
Epoch 183/200, Batch 16/17, Loss G: 4.024994850158691, Loss D: 0.01100226491689682
Epoch 184/200, Batch 0/17, Loss G: 3.6396121978759766, Loss D: 0.008321421220898628
Epoch 184/200, Batch 1/17, Loss G: 3.7274513244628906, Loss D: 0.004308268427848816
Epoch 184/200, Batch 2/17, Loss G: 3.9355626106262207, Loss D: 0.004213868174701929
Epoch 184/200, Batch 3/17, Loss G: 3.6975371837615967, Loss D: 0.0072224121

Epoch 189/200, Batch 5/17, Loss G: 3.691549777984619, Loss D: 0.004260813817381859
Epoch 189/200, Batch 6/17, Loss G: 3.703552722930908, Loss D: 0.0032613256480544806
Epoch 189/200, Batch 7/17, Loss G: 3.785841941833496, Loss D: 0.005295287352055311
Epoch 189/200, Batch 8/17, Loss G: 3.7671422958374023, Loss D: 0.005132950842380524
Epoch 189/200, Batch 9/17, Loss G: 3.5622129440307617, Loss D: 0.005456502083688974
Epoch 189/200, Batch 10/17, Loss G: 3.8470940589904785, Loss D: 0.003614844288676977
Epoch 189/200, Batch 11/17, Loss G: 3.72946834564209, Loss D: 0.005668548867106438
Epoch 189/200, Batch 12/17, Loss G: 3.797616958618164, Loss D: 0.0036944400053471327
Epoch 189/200, Batch 13/17, Loss G: 3.8892579078674316, Loss D: 0.002014534082263708
Epoch 189/200, Batch 14/17, Loss G: 3.7329931259155273, Loss D: 0.010457883588969707
Epoch 189/200, Batch 15/17, Loss G: 3.5214200019836426, Loss D: 0.010583865456283092
Epoch 189/200, Batch 16/17, Loss G: 3.6580686569213867, Loss D: 0.00864010

Epoch 195/200, Batch 2/17, Loss G: 3.6004881858825684, Loss D: 0.0330461747944355
Epoch 195/200, Batch 3/17, Loss G: 3.4497833251953125, Loss D: 0.04374327138066292
Epoch 195/200, Batch 4/17, Loss G: 3.646284818649292, Loss D: 0.06993553042411804
Epoch 195/200, Batch 5/17, Loss G: 3.5866384506225586, Loss D: 0.050352275371551514
Epoch 195/200, Batch 6/17, Loss G: 3.767062187194824, Loss D: 0.021037638187408447
Epoch 195/200, Batch 7/17, Loss G: 3.8171892166137695, Loss D: 0.04860841855406761
Epoch 195/200, Batch 8/17, Loss G: 3.5148725509643555, Loss D: 0.04315735772252083
Epoch 195/200, Batch 9/17, Loss G: 3.8933544158935547, Loss D: 0.016064129769802094
Epoch 195/200, Batch 10/17, Loss G: 3.7552223205566406, Loss D: 0.03129364177584648
Epoch 195/200, Batch 11/17, Loss G: 3.652827262878418, Loss D: 0.031634438782930374
Epoch 195/200, Batch 12/17, Loss G: 3.7642710208892822, Loss D: 0.019642436876893044
Epoch 195/200, Batch 13/17, Loss G: 3.7589125633239746, Loss D: 0.02454381808638572

Epoch 200/200, Batch 15/17, Loss G: 3.4882116317749023, Loss D: 0.006417832802981138
Epoch 200/200, Batch 16/17, Loss G: 3.775808334350586, Loss D: 0.0021065950859338045


In [31]:
import os

# Get the path to the user's desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Save the weights to the desktop
weights_path = os.path.join(desktop_path, "saved_weights5.pth")
torch.save(generator.state_dict(), weights_path)

In [38]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import os

# Load the trained model
generator = Generator()
weights_path = os.path.join(os.path.expanduser("~"), "Desktop", "saved_weights5.pth")
generator.load_state_dict(torch.load(weights_path))
generator.eval()

# Preprocess the test image
test_image_path = os.path.join(os.path.expanduser("~"), "Desktop" ,"onePieceColoring","black_and_white", "image_132.jpg")
image_size = (256, 256)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])
test_image = Image.open(test_image_path)
input_image = transform(test_image).unsqueeze(0)

# Pass the image through the model
with torch.no_grad():
    colorized_image = generator(input_image)

# Postprocess the colorized image
colorized_image = colorized_image.squeeze(0).permute(1, 2, 0)
colorized_image = (colorized_image + 1) / 2  # Denormalize the image

# Convert the tensor to a numpy array
colorized_image_np = colorized_image.cpu().numpy()

# Convert the numpy array to a PIL Image
colorized_image_pil = Image.fromarray((255 * colorized_image_np).astype(np.uint8))

# Save the colorized image
save_path = os.path.join(os.path.expanduser("~"), "Desktop", "colored_by_model5.jpg")
colorized_image_pil.save(save_path)